In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

taxiDB = pd.read_csv('taxi_dataset.csv')

In [ ]:
taxiDB.head(5)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N


<dl>
<dt> Описание колонок:
<dd>id - ID поездки </dd>
<dd>vendor_id - ID компании, осуществляющей перевозку </dd>
<dd>pickup_datetime - Таймкод начала поездки</dd>
<dd>dropoff_datetime - Таймкод конца поездки </dd>
<dd>passenger_count - Количество пассажиров </dd>
<dd>pickup_longitude - Долгота точки, в которой началась поездка </dd>
<dd>pickup_latitude - Широта точки, в которой началась поездка </dd>
<dd>dropoff_longitude - Долгота точки, в которой закончилась поездка </dd>
<dd>dropoff_latitude - Широта точки, в которой закончилась поездка </dd>
<dd>store_and_fwd_flag - Yes/No: Была ли информация сохранена в памяти транспортного средства из-за потери соединения с сервером </dd>
</dl>

Создаем таргет, в моем случаи это время поездки


In [ ]:
taxiDB['dropoff_datetime'] = pd.to_datetime(taxiDB['dropoff_datetime'])
taxiDB['pickup_datetime'] = pd.to_datetime(taxiDB['pickup_datetime'])
taxiDB['trip_duration'] = (taxiDB['dropoff_datetime'] - taxiDB['pickup_datetime']).dt.total_seconds()

Предсказывая таргет для новых объектов в будущем, мы не будем заранее знать **dropoff_datetime**.

Удалим колонку из датасета.

In [ ]:
taxiDB = taxiDB.drop(['dropoff_datetime'], axis=1)

Переведем vendor_id во множество {0, 1}

In [ ]:
taxiDB['vendor_id'] = taxiDB['vendor_id'] - 1

Переведем store_and_fwd_flag во множество {0, 1}

In [ ]:
taxiDB['store_and_fwd_flag'] = taxiDB['store_and_fwd_flag'].apply(lambda x: 0 if x == 'N' else 1)
taxiDB['store_and_fwd_flag'].value_counts()


store_and_fwd_flag
0    1450599
1       8045
Name: count, dtype: int64

Посчитаем Евклидово расстояние между точками подбора пассажира и точкой конца поездки.

In [ ]:
allLat  = list(taxiDB['pickup_latitude']) + list(taxiDB['dropoff_latitude'])
medianLat  = sorted(allLat)[int(len(allLat)/2)]
latMultiplier  = 111.32

taxiDB['pickup_latitude']   = latMultiplier  * (taxiDB['pickup_latitude']   - medianLat)
taxiDB['dropoff_latitude']   = latMultiplier  * (taxiDB['dropoff_latitude']  - medianLat)
allLong = list(taxiDB['pickup_longitude']) + list(taxiDB['dropoff_longitude'])
medianLong  = sorted(allLong)[int(len(allLong)/2)]
longMultiplier = np.cos(medianLat*(np.pi/180.0)) * 111.32
taxiDB['pickup_longitude']   = longMultiplier  * (taxiDB['pickup_longitude']   - medianLong)
taxiDB['dropoff_longitude']   = longMultiplier  * (taxiDB['dropoff_longitude']  - medianLong)
taxiDB['distance_km'] = ((taxiDB['dropoff_latitude'] - taxiDB['pickup_latitude'])**2 + (taxiDB['dropoff_longitude'] - taxiDB['pickup_longitude'])**2)**(1/2)
taxiDB.head(10)

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,distance_km
0,id2875421,1,2016-03-14 17:24:55,1,-0.110015,1.516008,1.367786,1.256121,0,455.0,1.500479
1,id2377394,0,2016-06-12 00:43:35,1,0.036672,-1.753813,-1.571088,-2.578912,0,663.0,1.807119
2,id3858529,1,2016-01-19 11:35:24,1,0.153763,1.070973,-2.064547,-4.923841,0,2124.0,6.392080
3,id3504673,1,2016-04-06 19:32:31,1,-2.461500,-3.823568,-2.649362,-5.298809,0,429.0,1.487155
4,id2181028,1,2016-03-26 13:30:55,1,0.657515,4.329328,0.668452,3.139453,0,435.0,1.189925
5,id0801584,1,2016-01-30 22:01:40,6,-0.169204,-1.349544,-0.947028,-0.571582,0,443.0,1.100107
6,id1813257,0,2016-06-17 22:34:59,4,0.997853,0.391954,1.977049,1.288819,0,341.0,1.327852
7,id1324603,1,2016-05-21 07:54:58,1,0.975979,4.838061,4.923001,0.694731,0,1551.0,5.722427
8,id1301050,0,2016-05-27 23:12:23,1,-1.571088,-1.772073,-0.416255,-2.393764,0,255.0,1.311541
9,id0012891,1,2016-03-10 21:45:01,1,-0.016727,-1.110890,0.662019,3.970922,0,1225.0,5.126939


Уберем старые признаки

In [ ]:
taxiDB = taxiDB.drop(['pickup_longitude', 'dropoff_longitude',
                      'pickup_latitude', 'dropoff_latitude'], axis=1)

Сохраним датасет с категориальными признаками для использования его в градиентном бустинге

In [ ]:
taxiDB_for_lgb = taxiDB.copy()
taxiDB_for_lgb = taxiDB_for_lgb.set_index('id')
taxiDB['passenger_count'].value_counts()

passenger_count
1    1033540
2     210318
5      78088
3      59896
6      48333
4      28404
0         60
7          3
9          1
8          1
Name: count, dtype: int64



Реализуем mean-target encoding, заменив колонку passenger_count колонкой category_encoded.

In [ ]:
taxiDB['passenger_count'] = taxiDB['passenger_count'].map(taxiDB.groupby(['passenger_count'])['trip_duration'].mean())
taxiDB = taxiDB.rename(columns={'passenger_count': 'category_encoded'})

In [ ]:
taxiDB = taxiDB.set_index('id')
taxiDB.head(10)

,vendor_id,pickup_datetime,category_encoded,store_and_fwd_flag,trip_duration,distance_km
id,,,,,,
id2875421,1,2016-03-14 17:24:55,930.399753,0,455.0,1.500479
id2377394,0,2016-06-12 00:43:35,930.399753,0,663.0,1.807119
id3858529,1,2016-01-19 11:35:24,930.399753,0,2124.0,6.392080
id3504673,1,2016-04-06 19:32:31,930.399753,0,429.0,1.487155
id2181028,1,2016-03-26 13:30:55,930.399753,0,435.0,1.189925
id0801584,1,2016-01-30 22:01:40,1061.355223,0,443.0,1.100107
id1813257,0,2016-06-17 22:34:59,1053.529749,0,341.0,1.327852
id1324603,1,2016-05-21 07:54:58,930.399753,0,1551.0,5.722427
id1301050,0,2016-05-27 23:12:23,930.399753,0,255.0,1.311541


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
X = taxiDB.drop(['trip_duration', 'pickup_datetime'], axis=1)
y = taxiDB['trip_duration']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

model=LinearRegression()
model.fit(X_train, y_train)
for column, coef in zip(X_train.columns, model.coef_):
    print(column, coef)
print(model.intercept_)
predicitions = model.predict(X_train)

vendor_id 192.14570788872823
category_encoded 0.28028146609588767
store_and_fwd_flag 57.32495444699315
distance_km 113.26084039511377
197.14834848566034


In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

features = X_test

y_pred = model.predict(features) # предсказанное значение
y_true = y_test # истинное значение


print("MAE = %s" % mean_absolute_error(
    y_pred, y_true)
)

mse = mean_squared_error(y_true, y_pred)

print('MSE = %s' % mse)

print("r2_score = %s" % r2_score(y_true, y_pred))

MAE = 466.459334983967
MSE = 10317626.622787377
r2_score = 0.025889266155565305


In [ ]:
df = pd.DataFrame(y_true.copy())
df['prediction'] = y_pred
df.head(20)

,trip_duration,prediction
id,,
id2793718,1040.0,963.364439
id3485529,827.0,872.218662
id1816614,614.0,848.528425
id1050851,867.0,895.462058
id0140657,4967.0,2854.024252
id2734699,374.0,748.328388
id2151697,1252.0,948.273854
id3635863,148.0,539.985658
id2169697,1499.0,1374.007118


Исходя из метрик можно сказать, что моя модель плохо предсказывает время поезди на такси в секундах, ошибаясь в среднем на 466с (MAE).